IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M4\FAULT_M4_5


VOLO m4 FAULT 5%

In [3]:
rcou_m4_fault5 = pd.read_csv("RCOU.csv")
rcou_m4_fault5 = rcou_m4_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

rcou_m4_fault5 = rcou_m4_fault5[((rcou_m4_fault5['C9'] >= 1400) & (rcou_m4_fault5['C10'] >= 1400) & (rcou_m4_fault5['C11'] >= 1400) & (rcou_m4_fault5['C12'] >= 1400) & (rcou_m4_fault5['C13'] >= 1400) & (rcou_m4_fault5['C14']>= 1400))]

pd.set_option('display.max_rows',700)
display(rcou_m4_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
225,121424667,1408,1411,1412,1407,1409,1410
228,121456481,1474,1408,1412,1471,1444,1440
229,121467192,1412,1492,1473,1433,1431,1474
234,121519915,1581,1430,1409,1597,1488,1531
235,121530107,1495,1549,1503,1542,1471,1571
...,...,...,...,...,...,...,...
5382,178639576,1630,1424,1638,1414,1489,1579
5389,178714326,1632,1403,1601,1444,1476,1575
5393,178757571,1615,1404,1594,1433,1469,1564
5394,178767858,1610,1406,1580,1446,1454,1573


In [4]:
xkf1_m4_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault5 = xkf1_m4_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault5 = xkf1_m4_fault5[((xkf1_m4_fault5['TimeUS'] >= 121424667) & (xkf1_m4_fault5['TimeUS'] <= 178858913))]
print(xkf1_m4_fault5)

         TimeUS  Roll  Pitch    Yaw
226   121432861  1.10   0.86  11.75
227   121442909  1.16   0.87  11.74
228   121453711  1.13   0.88  11.75
229   121464609  1.04   0.88  11.74
230   121475181  0.84   0.89  11.76
...         ...   ...    ...    ...
5398  178812690 -0.05   1.22   9.20
5399  178824912 -0.02   0.97   9.07
5400  178835655  0.00   0.79   8.93
5401  178846143  0.00   0.62   8.82
5402  178856096 -0.02   0.43   8.69

[5177 rows x 4 columns]


In [5]:
att_m4_fault5 = pd.read_csv("ATT.csv")
att_m4_fault5 = att_m4_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault5 = att_m4_fault5[((att_m4_fault5['TimeUS'] >= 121424667) & (att_m4_fault5['TimeUS'] <= 178858913))]
print(att_m4_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
226   121434103     0.29  1.10     -0.20   0.86   11.69  11.75
227   121444886     0.29  1.16     -0.20   0.87   11.69  11.74
228   121455563     0.29  1.13     -0.21   0.88   11.69  11.75
229   121466559     0.30  1.04     -0.22   0.88   11.69  11.74
230   121476723     0.31  0.84     -0.23   0.89   11.69  11.76
...         ...      ...   ...       ...    ...     ...    ...
5398  178813876     2.28 -0.05      0.76   1.22    6.95   9.20
5399  178826372     2.30 -0.02      0.60   0.97    6.94   9.07
5400  178837336     2.32  0.00      0.45   0.79    6.94   8.93
5401  178847331     2.35  0.00      0.31   0.62    6.93   8.82
5402  178858011     2.37 -0.02      0.19   0.43    6.93   8.69

[5177 rows x 7 columns]


In [6]:
att_m4_fault5 = pd.merge_asof(att_m4_fault5,xkf1_m4_fault5, on="TimeUS", by=["Roll","Pitch","Yaw"], tolerance=100000)
print(att_m4_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     121434103     0.29  1.10     -0.20   0.86   11.69  11.75
1     121444886     0.29  1.16     -0.20   0.87   11.69  11.74
2     121455563     0.29  1.13     -0.21   0.88   11.69  11.75
3     121466559     0.30  1.04     -0.22   0.88   11.69  11.74
4     121476723     0.31  0.84     -0.23   0.89   11.69  11.76
...         ...      ...   ...       ...    ...     ...    ...
5172  178813876     2.28 -0.05      0.76   1.22    6.95   9.20
5173  178826372     2.30 -0.02      0.60   0.97    6.94   9.07
5174  178837336     2.32  0.00      0.45   0.79    6.94   8.93
5175  178847331     2.35  0.00      0.31   0.62    6.93   8.82
5176  178858011     2.37 -0.02      0.19   0.43    6.93   8.69

[5177 rows x 7 columns]


In [7]:
esc_0_m4_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m4_fault5 = esc_0_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m4_fault5 = esc_0_m4_fault5[((esc_0_m4_fault5['TimeUS'] >= 121424667) & (esc_0_m4_fault5['TimeUS'] <= 178858913))]
print(esc_0_m4_fault5)

        TimeUS   RPM  Curr
22   121469298  3411  2.95
23   121575349  4155  4.28
24   121690084  5302  5.64
25   121797395  6127  5.49
26   121910041  6681  5.49
27   122014632  6641  5.39
28   122124134  6814  5.96
29   122245808  6812  5.69
30   122354583  6810  5.69
31   122462508  6920  5.48
32   122571050  6982  6.24
33   122673092  6956  5.96
34   122780614  6950  6.04
35   122885294  7054  5.90
36   122991332  6971  6.47
37   123434108  6876  3.86
38   123547497  6283  4.09
39   123647936  6267  3.56
40   123754264  6078  3.24
41   123871268  5915  3.55
42   123971614  5787  3.73
43   124181926  5969  3.79
44   124299060  5880  3.99
45   124412877  6020  3.86
46   124513705  6092  4.26
47   124621092  6174  4.80
48   124739063  6460  5.26
49   124850943  6500  4.40
50   124947844  6507  4.29
51   125065080  6268  4.59
52   125172798  6379  4.16
53   125281010  6327  4.26
54   125388316  6308  4.04
55   125498575  6196  4.01
56   125614533  6105  3.93
57   125713019  6069  4.14
5

In [8]:
imu_2_m4_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault5 = imu_2_m4_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault5 = imu_2_m4_fault5[((imu_2_m4_fault5['TimeUS'] >= 121424667) & (imu_2_m4_fault5['TimeUS'] <= 178858913))]
print(imu_2_m4_fault5)

          TimeUS      GyrX      GyrY      GyrZ      AccX      AccY      AccZ
891    121426347 -0.064619 -0.007186 -0.001358  0.132007 -0.119005 -9.908481
892    121429392 -0.047707  0.009100  0.005156  0.169139 -0.053444 -9.823930
893    121431996  0.101188  0.005335  0.002265  0.214895  0.043437 -9.632129
894    121433935  0.133678 -0.005134 -0.000419  0.190652  0.090394 -9.689696
895    121437323 -0.037317 -0.000011  0.002578  0.180219  0.111405 -9.752580
...          ...       ...       ...       ...       ...       ...       ...
20555  178847110 -0.045952 -0.221907 -0.193600  0.601780  0.184901 -9.610149
20556  178850053  0.007346 -0.264784 -0.199028  0.633652  0.164161 -9.280994
20557  178852370  0.026327 -0.401838 -0.218364  0.543415  0.137789 -9.337992
20558  178854843 -0.010647 -0.342378 -0.218698  0.401039  0.143443 -9.645085
20559  178857906 -0.072953 -0.236675 -0.212978  0.248086  0.150457 -9.989474

[19669 rows x 7 columns]


In [9]:
m4_fault5 = pd.merge_asof(att_m4_fault5, imu_2_m4_fault5, on="TimeUS",  tolerance= 100000)
print(m4_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw      GyrX  \
0     121434103     0.29  1.10     -0.20   0.86   11.69  11.75  0.133678   
1     121444886     0.29  1.16     -0.20   0.87   11.69  11.74  0.114906   
2     121455563     0.29  1.13     -0.21   0.88   11.69  11.75  0.032570   
3     121466559     0.30  1.04     -0.22   0.88   11.69  11.74 -0.185548   
4     121476723     0.31  0.84     -0.23   0.89   11.69  11.76 -0.293293   
...         ...      ...   ...       ...    ...     ...    ...       ...   
5172  178813876     2.28 -0.05      0.76   1.22    6.95   9.20 -0.059856   
5173  178826372     2.30 -0.02      0.60   0.97    6.94   9.07 -0.045775   
5174  178837336     2.32  0.00      0.45   0.79    6.94   8.93 -0.033969   
5175  178847331     2.35  0.00      0.31   0.62    6.93   8.82 -0.045952   
5176  178858011     2.37 -0.02      0.19   0.43    6.93   8.69 -0.072953   

          GyrY      GyrZ      AccX      AccY       AccZ  
0    -0.005134 -0.000419  0.1

In [10]:
m4_fault5 = pd.merge_asof(esc_0_m4_fault5, m4_fault5, on="TimeUS",  tolerance= 100000)
print(m4_fault5)

        TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw  \
0    121469298  3411  2.95     0.30   1.04     -0.22   0.88   11.69   11.74   
1    121575349  4155  4.28     0.20   1.05     -0.33   1.10   11.69   11.84   
2    121690084  5302  5.64    -0.15   1.45     -0.29   0.74   11.69   12.31   
3    121797395  6127  5.49    -0.92   2.17     -0.29   0.39   11.69   13.32   
4    121910041  6681  5.49    -2.16   2.50      0.00  -0.29   11.68   14.54   
5    122014632  6641  5.39    -3.56   1.63      0.51  -0.38   11.65   15.75   
6    122124134  6814  5.96    -5.37  -0.13      1.07   0.36   11.61   16.85   
7    122245808  6812  5.69    -7.73  -2.60      1.42   1.10   11.57   18.22   
8    122354583  6810  5.69    -9.56  -4.88      1.65   1.59   11.53   19.38   
9    122462508  6920  5.48   -10.80  -7.10      1.41   2.67   11.58   19.88   
10   122571050  6982  6.24   -11.79  -8.30      1.06   2.75   11.62   20.22   
11   122673092  6956  5.96   -12.52  -9.02      0.75

In [11]:
m4_fault5["TimeUS"] = m4_fault5["TimeUS"] - m4_fault5.iloc[0]["TimeUS"]
m4_fault5["TimeUS"] = m4_fault5["TimeUS"].astype(int)
m4_fault5["Guasto"] = 1
print(m4_fault5)

       TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw  \
0           0  3411  2.95     0.30   1.04     -0.22   0.88   11.69   11.74   
1      106051  4155  4.28     0.20   1.05     -0.33   1.10   11.69   11.84   
2      220786  5302  5.64    -0.15   1.45     -0.29   0.74   11.69   12.31   
3      328097  6127  5.49    -0.92   2.17     -0.29   0.39   11.69   13.32   
4      440743  6681  5.49    -2.16   2.50      0.00  -0.29   11.68   14.54   
5      545334  6641  5.39    -3.56   1.63      0.51  -0.38   11.65   15.75   
6      654836  6814  5.96    -5.37  -0.13      1.07   0.36   11.61   16.85   
7      776510  6812  5.69    -7.73  -2.60      1.42   1.10   11.57   18.22   
8      885285  6810  5.69    -9.56  -4.88      1.65   1.59   11.53   19.38   
9      993210  6920  5.48   -10.80  -7.10      1.41   2.67   11.58   19.88   
10    1101752  6982  6.24   -11.79  -8.30      1.06   2.75   11.62   20.22   
11    1203794  6956  5.96   -12.52  -9.02      0.75   2.30   10.

In [12]:
path_file = path_file.replace(r"m4\FAULT_m4_5", "")
os.chdir(path_file)
m4_fault5.to_csv('m4_fault5.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M4\FAULT_M4_5
